# Metadata

**L1 Taxonomy** - Software Architecture & Design

**L2 Taxonomy** - Singleton Pattern

**Subtopic** - Singleton used for configuration: a Config class reads file once globally accessible one instance

**Use Case** - Implement a Config singleton class in Python that reads a JSON configuration file once and provides globally accessible configuration parameters. The class should be able to handle the absence or unavailability of the configuration file and provide default configuration parameters in such cases.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
```


# Prompt

# Problem Description

You need a thread-safe Singleton `Config` class that builds its settings from four layers provided entirely via JSON input (no file or real environment access):

1. **Defaults**:  a built-in dict.
2. **Config JSON** : the contents of what would have been `config.json`.
3. **Environment overrides** : key/value pairs simulating `APP_` env vars.
4. **Command-line overrides** : highest-priority dict.

The class must support nested keys via dot-notation (e.g. `"database.host"`) and provide read-only methods (`get`, `get_int`, `get_bool`). Reads must be lock-free; all merges and a `reload()` must be guarded by an `RLock`.

# Input Format

All inputs come as one JSON object, for example:

```json
{
  "defaults": {
    "server": { "host": "127.0.0.1", "port": 8000 },
    "logging": { "level": "INFO" }
  },
  "config_json": {
    "server": { "port": 9000 },
    "feature_flags": { "beta": true }
  },
  "env_vars": {
    "server__host": "0.0.0.0",
    "feature_flags__beta": "false"
  },
  "cli_overrides": {
    "server__port": 8081,
    "new_feature": true
  }
}
```

# Output Format

The `Config` singleton exposes:

```python
cfg.get("server.host")            # "0.0.0.0"
cfg.get_int("server.port")        # 8081
cfg.get_bool("feature_flags.beta")# False
cfg.get("new_feature")            # True
```

# Examples

Given the above input, the final merged config is:

```json
{
  "server": { "host": "0.0.0.0", "port": 8081 },
  "logging": { "level": "INFO" },
  "feature_flags": { "beta": false },
  "new_feature": true
}
```


# Requirements


# Explicit Requirements

1. Singleton accessed via:

   ```python
   cfg = Config.get_instance(input_json)
   ```
2. Merge order: defaults < config_json < env_vars < cli_overrides.
3. Use `threading.RLock` for all writes and `reload()`.
4. Reads (`get`, `get_int`, `get_bool`) must never acquire the lock.
5. Support nested lookups with dot-notation.
6. Type helpers must safely cast or return the provided default.
7. `reload()` re-applies layers 2–3 (preserving cli overrides) under lock.

# Implicit Requirements

* Values may be str, int, float, bool, list, or dict.
* Env var names use `__` to denote nesting and always override JSON, even if empty string.
* CLI overrides are permanent and survive `reload()`.
* Unknown keys in JSON or env must be added dynamically.
* Keys are case-sensitive.

# Function Signature

```python
class Config:
    @classmethod
    def get_instance(cls, input_json: dict) -> "Config": ...
    def get(self, key: str, default=None) -> Any: ...
    def get_int(self, key: str, default: int | None = None) -> int | None: ...
    def get_bool(self, key: str, default: bool | None = None) -> bool | None: ...
    def reload(self) -> None: ...
```

# Constraints

* Total merge time ≤ 50 ms.
* Memory footprint ≤ 5 MB.
* Concurrent `reload()` calls never raise or return partial data.
* Reader threads block < 1 ms during reload.
* All override keys must be valid identifiers or use `__`.

# Edge Cases

* `config_json` is `null` or empty.
* `config_json` contains malformed types.
* Env var refers to missing nested path (auto-create).
* CLI override supplies scalar where dict expected.
* Simultaneous `get_instance()` calls from many threads.

# Solution Expectation

Implement lazy instantiation in `get_instance()` guarded by an `RLock`. Internally keep:

* `_base` = deep copy of `defaults`
* `_json_layer` = deep merge of `_base` + `config_json`
* `_env_layer` = deep merge of `_json_layer` + parsed `env_vars`
* `_overrides` = `cli_overrides` (always reapplied last)

On startup: Build `_config = merge(_env_layer, _overrides)`.
- `get()` splits `key` on `"."` and walks `_config`.
- `reload()` acquires lock, rebuilds `_json_layer` and `_env_layer` from input
JSON, merges `_config = merge(_env_layer, _overrides)`, then releases lock.
Provide a private `_deep_merge(dest, src)` that recursively merges dicts.



In [1]:
# code

import threading
from typing import Any, Dict, Optional


class Config:
    """Thread-safe Singleton configuration with layered overrides."""

    _instance: Optional["Config"] = None
    _lock = threading.RLock()

    def __init__(self, input_json: Dict[str, Any]) -> None:
        """
        Build configuration by merging defaults, JSON, env vars, and CLI overrides.

        Args:
            input_json: Dict with keys "defaults", "config_json",
                        "env_vars", and "cli_overrides".
        """
        self._defaults = input_json.get("defaults", {})
        self._json_data = input_json.get("config_json", {})
        self._raw_env = input_json.get("env_vars", {})
        self._raw_cli = input_json.get("cli_overrides", {})

        # Parse nested structures from flat env and CLI
        self._env_layer = self._parse_nested(self._raw_env)
        self._cli_layer = self._parse_nested(self._raw_cli)

        # Initial merge under lock
        with self._lock:
            self._rebuild_config()

    @classmethod
    def get_instance(cls, input_json: Dict[str, Any]) -> "Config":
        """
        Return the singleton instance, creating it on first call.

        Args:
            input_json: Same dict passed on first invocation.
        """
        with cls._lock:
            if cls._instance is None:
                cls._instance = cls(input_json)
        return cls._instance

    def _deep_merge(self, dest: Dict[str, Any],
                    src: Dict[str, Any]) -> Dict[str, Any]:
        """
        Recursively merge src into dest and return dest.

        Args:
            dest: Destination dict to be updated.
            src: Source dict whose values override dest.
        """
        for key, value in src.items():
            if (
                key in dest
                and isinstance(dest[key], dict)
                and isinstance(value, dict)
            ):
                dest[key] = self._deep_merge(dest[key], value)
            else:
                dest[key] = value
        return dest

    def _parse_nested(self, flat: Dict[str, Any]) -> Dict[str, Any]:
        """
        Expand keys containing '__' into nested dicts.

        Example: {'a__b': 1} -> {'a': {'b': 1}}
        """
        result: Dict[str, Any] = {}
        for flat_key, val in flat.items():
            parts = flat_key.split("__")
            node = result
            for part in parts[:-1]:
                node = node.setdefault(part, {})
            node[parts[-1]] = val
        return result

    def _rebuild_config(self) -> None:
        """
        Merge all layers into the internal _config dict.
        Called under _lock.
        """
        cfg = {}
        self._deep_merge(cfg, self._defaults)
        self._deep_merge(cfg, self._json_data)
        self._deep_merge(cfg, self._env_layer)
        self._deep_merge(cfg, self._cli_layer)
        self._config = cfg

    def get(self, key: str, default: Any = None) -> Any:
        """
        Retrieve a value by dot-notation key.

        Args:
            key: e.g. "section.sub.key"
            default: returned if key is missing
        """
        node = self._config
        for part in key.split("."):
            if not isinstance(node, dict) or part not in node:
                return default
            node = node[part]
        return node

    def get_int(self, key: str, default: Optional[int] = None
                ) -> Optional[int]:
        """
        Retrieve an integer, casting floats or numeric strings.

        Returns default on failure.
        """
        val = self.get(key, default)
        if isinstance(val, (int, float)):
            return int(val)
        try:
            return int(val)  # type: ignore
        except Exception:
            return default

    def get_bool(self, key: str, default: Optional[bool] = None
                 ) -> Optional[bool]:
        """
        Retrieve a boolean. Accepts True/False or "true"/"false" strings.

        Returns default on failure.
        """
        val = self.get(key, default)
        if isinstance(val, bool):
            return val
        if isinstance(val, str):
            low = val.lower()
            if low in ("true", "false"):
                return low == "true"
        return default

    def reload(self) -> None:
        """
        Re-apply config_json and env_vars under lock, preserving CLI overrides.
        """
        with self._lock:
            self._json_data = self._json_data.copy()
            self._env_layer = self._parse_nested(self._raw_env)
            self._rebuild_config()



In [ ]:
# tests


import unittest
import threading
from copy import deepcopy

# Assume Config is imported from config_module
from main import Config


class TestConfig(unittest.TestCase):
    """Unit tests for the Config singleton."""

    def setUp(self) -> None:
        """Prepare fresh input data for each test run."""
        self.base_input = {
            "defaults": {
                "server": {"host": "127.0.0.1", "port": 8000},
                "logging": {"level": "INFO"},
            },
            "config_json": {
                "server": {"port": 9000},
                "feature_flags": {"beta": True},
            },
            "env_vars": {
                "server__host": "0.0.0.0",
                "feature_flags__beta": "false",
            },
            "cli_overrides": {
                "server__port": 8081,
                "new_feature": True,
            },
        }
        # Reset singleton between tests
        Config._instance = None  # pylint: disable=protected-access

    def test_singleton_identity(self) -> None:
        """Two get_instance calls should return exactly the same object."""
        cfg1 = Config.get_instance(deepcopy(self.base_input))
        cfg2 = Config.get_instance(deepcopy(self.base_input))
        self.assertIs(cfg1, cfg2)

    def test_merge_order_and_values(self) -> None:
        """Verify the final merged output respects layer priorities."""
        cfg = Config.get_instance(deepcopy(self.base_input))

        # CLI override wins
        self.assertEqual(cfg.get_int("server.port"), 8081)
        # Env var overrides config_json
        self.assertEqual(cfg.get("server.host"), "0.0.0.0")
        # config_json overrides defaults
        self.assertFalse(cfg.get_bool("feature_flags.beta"))
        # Default remains when not overridden
        self.assertEqual(cfg.get("logging.level"), "INFO")
        # New key from CLI is present
        self.assertTrue(cfg.get_bool("new_feature"))

    def test_get_helpers_and_defaults(self) -> None:
        """Check get_int and get_bool conversions and default fallbacks."""
        cfg = Config.get_instance(deepcopy(self.base_input))

        # Numeric string converts to int
        cfg._config["threshold"] = "42"  # pylint: disable=protected-access
        self.assertEqual(cfg.get_int("threshold"), 42)

        # Non-numeric string falls back
        self.assertIsNone(cfg.get_int("logging.level"))

        # Boolean string handled
        cfg._config["flag"] = "true"  # pylint: disable=protected-access
        self.assertTrue(cfg.get_bool("flag"))

        # Missing key returns supplied default
        self.assertEqual(cfg.get("missing.key", "NA"), "NA")

    def test_reload_updates_json_and_env_only(self) -> None:
        """reload should pick up new JSON/env layers but keep CLI overrides."""
        cfg = Config.get_instance(deepcopy(self.base_input))

        # Simulate changes
        cfg._json_data["server"]["port"] = 9500  # pylint: disable=protected-access
        cfg._raw_env["server__host"] = "10.0.0.1"  # pylint: disable=protected-access

        cfg.reload()

        # CLI override still wins
        self.assertEqual(cfg.get_int("server.port"), 8081)
        # Updated env var visible
        self.assertEqual(cfg.get("server.host"), "10.0.0.1")
        # JSON change visible when not trumped by CLI
        self.assertEqual(cfg.get_int("server.port"), 8081)  # still overridden
        self.assertEqual(cfg.get("logging.level"), "INFO")  # unchanged

    def test_thread_safety_of_get(self) -> None:
        """Ensure concurrent reads do not raise and return consistent values."""

        cfg = Config.get_instance(deepcopy(self.base_input))
        results = []

        def reader() -> None:
            for _ in range(10_000):
                host = cfg.get("server.host")
                port = cfg.get_int("server.port")
                results.append((host, port))

        threads = [threading.Thread(target=reader) for _ in range(4)]
        for t_thr in threads:
            t_thr.start()
        for t_thr in threads:
            t_thr.join()

        # All reads should be consistent with current config state
        expected = ("0.0.0.0", 8081)
        self.assertTrue(all(pair == expected for pair in results))


if __name__ == "__main__":
    unittest.main()


# Model Breaking Proof

#### Model Breaking Task URL: <Add the URL here>

#### Model code:

```python
import threading
from typing import Any, Union, Optional
import copy

class Config:
    _instance = None
    _lock = threading.RLock()

    def __init__(self, input_json: dict):
        self._input_json = input_json
        self._base = copy.deepcopy(input_json.get("defaults", {}))
        self._json_layer = self._deep_merge(copy.deepcopy(self._base), input_json.get("config_json", {}))
        self._env_layer = self._deep_merge(copy.deepcopy(self._json_layer), self._parse_env_vars(input_json.get("env_vars", {})))
        # <Issue>: CLI overrides are taken as a flat dict, not parsed into nested structure via '__'
        self._overrides = input_json.get("cli_overrides", {})
        # <Issue>: Merging flat CLI overrides directly; nested keys using '__' won’t be expanded
        self._config = self._deep_merge(copy.deepcopy(self._env_layer), self._overrides)

    @classmethod
    def get_instance(cls, input_json: dict) -> 'Config':
        if cls._instance is None:
            with cls._lock:
                if cls._instance is None:
                    cls._instance = cls(input_json)
        return cls._instance

    def _deep_merge(self, dest: dict, src: dict) -> dict:
        for key, value in src.items():
            if isinstance(value, dict) and key in dest and isinstance(dest[key], dict):
                dest[key] = self._deep_merge(dest[key], value)
            else:
                dest[key] = value
        return dest

    def _parse_env_vars(self, env_vars: dict) -> dict:
        result = {}
        for key, value in env_vars.items():
            keys = key.split('__')
            current = result
            for part in keys[:-1]:
                if part not in current or not isinstance(current[part], dict):
                    current[part] = {}
                current = current[part]
            current[keys[-1]] = self._parse_value(value)
        return result

    def _parse_value(self, value: str) -> Union[str, bool]:
        if value.lower() == 'true':
            return True
        elif value.lower() == 'false':
            return False
        return value

    def get(self, key: str, default: Optional[Any] = None) -> Any:
        parts = key.split('.')
        current = self._config
        for part in parts:
            if isinstance(current, dict) and part in current:
                current = current[part]
            else:
                return default
        return current

    def get_int(self, key: str, default: Optional[int] = None) -> Optional[int]:
        value = self.get(key, default)
        if isinstance(value, (int, float)):
            return int(value)
        return default

    def get_bool(self, key: str, default: Optional[bool] = None) -> Optional[bool]:
        value = self.get(key, default)
        if isinstance(value, bool):
            return value
        elif isinstance(value, str):
            return value.lower() == 'true'
        return default

    def reload(self) -> None:
        with self._lock:
            # <Issue>: reload() is incomplete and has a syntax error; does not rebuild layers or swap _config
            self._json_layer = self._deep_merge(copy))

```